Flow for a machine learning project

1. Load configuration
2. Initialize data processor
3. Load and preprocess housing data
4. For each edge connectivity method:
   a. Build graph with specific connectivity
   b. For each model type (Simple/Multi-layer):
      - Initialize model
      - Train model
      - Evaluate on validation set
      - Save results
5. Compare all results
6. Get test result and Generate final report

In [ ]:
#data pipeline

In [ ]:
#configuring for experiments

In [ ]:
#running experiments

In [ ]:
#evaluating results